# Grupo 3: Trabajo Final Integrador

Churichi Alan, Gonzalez Exequiel, Mare Rodrigo

In [1]:
import roboticstoolbox as rtb
import numpy as np
import sim
import sys
import time
from handlers import *
from kinematics import *
from utils import *
from trajectories import *
from constants import *

### Movimiento del robot utilizando el MCI calculado

In [2]:
clientID = connect(19999)

t0 = 0;
if (clientID != -1): # Mientras haya conexión con Coppelia
    joints = [0 for i in range(JOINTS_NUM)] # ID de las articulaciones
    home = {}
    checkpoints_traj_1 = [{} for i in range(CHECKPOINTS_TRAJ_1_NUM)]
    checkpoints_pioneer = [{} for i in range(CHECKPOINTS_PIONEER_NUM)]
    checkpoints_boxes = [{} for i in range(CHECKPOINTS_BOXES_NUM)]
    
    # Se obtienen los IDs
    joint_handlers(joints, JOINTS_NUM, clientID, sim.simx_opmode_blocking)
    checkpoint_handlers(checkpoints_traj_1, 'CheckPoint', CHECKPOINTS_TRAJ_1_NUM, clientID, sim.simx_opmode_blocking)
    checkpoint_handlers(checkpoints_pioneer, 'CheckPoint_box_pioneer', CHECKPOINTS_PIONEER_NUM, clientID, sim.simx_opmode_blocking)
    checkpoint_handlers(checkpoints_boxes, 'CheckPoint_box', CHECKPOINTS_BOXES_NUM, clientID, sim.simx_opmode_blocking)
    _, tip = sim.simxGetObjectHandle(clientID, (ROBOT_NAME + '_tip'), sim.simx_opmode_blocking)
    _, sensor_1 = sim.simxGetObjectHandle(clientID, 'Proximity_sensor_box_low', sim.simx_opmode_blocking)
    _, sensor_2 = sim.simxGetObjectHandle(clientID, 'Proximity_sensor_box_high', sim.simx_opmode_blocking)
    
    # Se setean las posiciones y orientaciones como streaming
    sim.simxGetObjectPosition(clientID, tip, -1, sim.simx_opmode_streaming)
    sim.simxGetObjectOrientation(clientID, tip, -1, sim.simx_opmode_streaming)
    checkpoints_set_streaming([checkpoints_traj_1, checkpoints_pioneer, checkpoints_boxes], clientID)
    
    sim.simxReadProximitySensor(clientID, sensor_1, sim.simx_opmode_streaming)
    sim.simxReadProximitySensor(clientID, sensor_2, sim.simx_opmode_streaming)

    # Se inicia la simulación
    sim.simxStartSimulation(clientID, sim.simx_opmode_oneshot)
    # sim.simxPauseSimulation(clientID, sim.simx_opmode_oneshot)
    
    time.sleep(1) # Tiempo de espera para asegurarse que los valores leidos de home sean correctos
    
    # Posición, orientación y ángulos de Home
    _, home['position'] = sim.simxGetObjectPosition(clientID, tip, -1, sim.simx_opmode_buffer)
    _, home['orientation'] = sim.simxGetObjectOrientation(clientID, tip, -1, sim.simx_opmode_buffer)
    home['angles'] = inverse_kinematics(home['position'], home['orientation'])
    
    # Posición, orientación y ángulos de Checkpoints
    checkpoints_get_from_buffer([checkpoints_traj_1, checkpoints_pioneer, checkpoints_boxes], clientID)
        
    points_fast_translation = 50  # Movimiento rápido
    points_mid_translation = 150  # Movimiento moderado
    points_slow_translation = 250 # Movimiento lento
    
    print('Calculando trayectorias...')

    get_large_box = get_angles_for_trajectory([home, checkpoints_traj_1[1], checkpoints_boxes[1]], ['jtraj', 'ctraj'], points_fast_translation)
    move_large_box = get_angles_for_trajectory([checkpoints_boxes[1], checkpoints_traj_1[1], checkpoints_traj_1[0]], ['ctraj', 'ctraj'], points_mid_translation)
    store_large_box_pioneer = []
    store_large_box_pioneer.append(get_angles_for_trajectory([checkpoints_traj_1[0], checkpoints_pioneer[0]], ['ctraj'], points_mid_translation))
    store_large_box_pioneer.append(get_angles_for_trajectory([checkpoints_traj_1[0], checkpoints_pioneer[2]], ['ctraj'], points_mid_translation))
    store_large_box_pioneer.append(get_angles_for_trajectory([checkpoints_traj_1[0], checkpoints_pioneer[4]], ['ctraj'], points_mid_translation))
    back_home_large_box = []
    back_home_large_box.append(get_angles_for_trajectory([checkpoints_pioneer[0], checkpoints_traj_1[0], home], ['jtraj', 'jtraj'], points_fast_translation))
    back_home_large_box.append(get_angles_for_trajectory([checkpoints_pioneer[2], checkpoints_traj_1[0], home], ['jtraj', 'jtraj'], points_fast_translation))
    back_home_large_box.append(get_angles_for_trajectory([checkpoints_pioneer[4], checkpoints_traj_1[0], home], ['jtraj', 'jtraj'], points_fast_translation))

    get_small_box = get_angles_for_trajectory([home, checkpoints_traj_1[1], checkpoints_boxes[0]], ['jtraj', 'ctraj', 'ctraj'], points_fast_translation)
    move_small_box = get_angles_for_trajectory([checkpoints_boxes[0], checkpoints_traj_1[1], checkpoints_traj_1[0]], ['ctraj', 'ctraj'], points_mid_translation)
    store_small_box_pioneer = []
    store_small_box_pioneer.append(get_angles_for_trajectory([checkpoints_traj_1[0], checkpoints_pioneer[1]], ['ctraj'], points_mid_translation))
    store_small_box_pioneer.append(get_angles_for_trajectory([checkpoints_traj_1[0], checkpoints_pioneer[3]], ['ctraj'], points_mid_translation))
    store_small_box_pioneer.append(get_angles_for_trajectory([checkpoints_traj_1[0], checkpoints_pioneer[5]], ['ctraj'], points_mid_translation))
    back_home_small_box = []
    back_home_small_box.append(get_angles_for_trajectory([checkpoints_pioneer[1], checkpoints_traj_1[0], home], ['jtraj', 'jtraj'], points_fast_translation))
    back_home_small_box.append(get_angles_for_trajectory([checkpoints_pioneer[3], checkpoints_traj_1[0], home], ['jtraj', 'jtraj'], points_fast_translation))
    back_home_small_box.append(get_angles_for_trajectory([checkpoints_pioneer[5], checkpoints_traj_1[0], home], ['jtraj', 'jtraj'], points_fast_translation))
    
    sim.simxSetFloatSignal(clientID, 'beltVelocity', 0.5, sim.simx_opmode_oneshot)
    
    print('Trayectorias listas')
    print('Iniciando simulación')

    
    for i in range(BOX_COUNT):
        sensor_1_value = False
        while (sensor_1_value == False):
            _, sensor_1_value, _, _, _ = sim.simxReadProximitySensor(clientID, sensor_1, sim.simx_opmode_buffer)
            _, sensor_2_value, _, _, _ = sim.simxReadProximitySensor(clientID, sensor_2, sim.simx_opmode_streaming)
        
        sim.simxSetFloatSignal(clientID, 'beltVelocity', 0, sim.simx_opmode_oneshot)

        if sensor_2_value: # Caja grande
            for traj_point in get_large_box:
                time.sleep(0.05)
                setJointsPosition(traj_point, joints, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)
            
            sim.simxSetIntegerSignal(clientID, 'VacuumGripper_active', 1, sim.simx_opmode_oneshot) # Agarrar

            for traj_point in move_large_box:
                time.sleep(0.05)
                setJointsPosition(traj_point, joints, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)

            for traj_point in store_large_box_pioneer[i]:
                time.sleep(0.05)
                setJointsPosition(traj_point, joints, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)

            sim.simxSetIntegerSignal(clientID, 'VacuumGripper_active', 0, sim.simx_opmode_oneshot) # Soltar

            for traj_point in back_home_large_box[i]:
                time.sleep(0.05)
                setJointsPosition(traj_point, joints, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)
        else: # Caja chica
            for traj_point in get_small_box:
                time.sleep(0.05)
                setJointsPosition(traj_point, joints, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)
            
            sim.simxSetIntegerSignal(clientID, 'VacuumGripper_active', 1, sim.simx_opmode_oneshot) # Agarrar

            for traj_point in move_small_box:
                time.sleep(0.05)
                setJointsPosition(traj_point, joints, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)
                
            for traj_point in store_small_box_pioneer[i]:
                time.sleep(0.05)
                setJointsPosition(traj_point, joints, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)

            sim.simxSetIntegerSignal(clientID, 'VacuumGripper_active', 0, sim.simx_opmode_oneshot) # Soltar

            for traj_point in back_home_small_box[i]:
                time.sleep(0.05)
                setJointsPosition(traj_point, joints, JOINTS_NUM, clientID, sim.simx_opmode_oneshot)

        sim.simxSetFloatSignal(clientID, 'beltVelocity', 0.5, sim.simx_opmode_oneshot)   

    

    # sim.simxSetIntegerSignal(clientID, 'VacuumGripper_active', 1, sim.simx_opmode_oneshot) # Agarrar

    sim.simxFinish(clientID);
    print('Conexión Finalizada')



Conectado a  19999
Calculando trayectorias...
Trayectorias listas
Iniciando simulación
Conexión Finalizada
